In [17]:
import pandas as pd
import numpy as np

In [18]:
path = "../Data/Online_Data/trainning_data/corporate_attributes/"
outpath = "../Processed_Data/"

In [19]:
company_ar_alterstockinfo = pd.read_csv(path+"company_ar_alterstockinfo.csv")

In [20]:
company_ar_alterstockinfo
company_ar_alterstockinfo = company_ar_alterstockinfo.drop(["ALTDATE"],axis=1)
company_ar_alterstockinfo["TRANSAMPR"] = company_ar_alterstockinfo["TRANSAMPR"].fillna(0)
company_ar_alterstockinfo["TRANSAMAFT"] = company_ar_alterstockinfo["TRANSAMAFT"].fillna(0)
company_ar_alterstockinfo

,entid,ANCHEYEAR,INV,TRANSAMPR,TRANSAMAFT
0,959830672,2014,37,0.162185,101.969149
1,959830672,2014,294,100.905411,-0.640097
2,658987796,2014,357,59.872529,0.586056
3,658987796,2014,316,-0.075377,-0.664907
4,658987796,2014,290,0.419384,9.575938
...,...,...,...,...,...
3674,708549128,2018,1224,-1.948806,30.929302
3675,708549128,2018,1961,100.759688,70.083031
3676,791477068,2018,1203,18.652548,0.237149
3677,791477068,2018,2457,1.967559,20.661326


In [21]:
df_inv = company_ar_alterstockinfo.drop(["TRANSAMPR","TRANSAMAFT"],axis=1).copy()
df_inv = df_inv.groupby(['entid', 'ANCHEYEAR']).count()
df_inv = df_inv.groupby(['entid']).mean()
df_inv

,INV
entid,
243165,2.0
683429,2.0
703531,1.5
2126661,2.0
3926205,2.0
...,...
1223416010,5.0
1225572865,3.0
1227102837,1.0


In [22]:
df_inv_unq = company_ar_alterstockinfo.drop(["TRANSAMPR","TRANSAMAFT"],axis=1).drop_duplicates().copy()
df_inv_unq = df_inv_unq.groupby(['entid', 'ANCHEYEAR']).count()
df_inv_unq = df_inv_unq.groupby(['entid']).mean()
df_inv_unq

,INV
entid,
243165,2.0
683429,2.0
703531,1.0
2126661,2.0
3926205,2.0
...,...
1223416010,5.0
1225572865,3.0
1227102837,1.0


In [23]:
company_ar_alterstockinfo["net_change_per_year"] = company_ar_alterstockinfo["TRANSAMAFT"] - company_ar_alterstockinfo["TRANSAMPR"]
df_altinfo = company_ar_alterstockinfo.drop(["INV","TRANSAMPR","TRANSAMAFT"],axis=1).copy()
df_altinfo = df_altinfo.groupby(['entid', 'ANCHEYEAR']).sum()
df_altinfo = df_altinfo.groupby(['entid']).mean()
df_altinfo

,net_change_per_year
entid,
243165,0.621989
683429,-103.595820
703531,1.930562
2126661,0.262459
3926205,-4.049364
...,...
1223416010,-4.278679
1225572865,5.526924
1227102837,-49.806573


In [31]:
company_ar_alterstockinfo["vol_year"] = company_ar_alterstockinfo["TRANSAMAFT"] -  company_ar_alterstockinfo["TRANSAMPR"]
df_vol = company_ar_alterstockinfo.drop(["INV","TRANSAMPR","TRANSAMAFT","net_change_per_year"],axis=1).copy()
df_vol = df_vol.groupby(['entid', 'ANCHEYEAR']).sum()
df_vol = df_vol.groupby(['entid']).mean()

df_vol["vol_year"] = df_vol["vol_year"].values/df_inv["INV"].values
df_vol

,vol_year
entid,
243165,0.310994
683429,-51.797910
703531,1.287041
2126661,0.131229
3926205,-2.024682
...,...
1223416010,-0.855736
1225572865,1.842308
1227102837,-49.806573


In [32]:
df = pd.DataFrame()
df["entid"] = df_vol.iloc[:,0] = df_inv.index.values
df["Vol/transaction_per_year"] = df_vol.iloc[:,0].values
df["net_change_per_year"] = df_altinfo.iloc[:,0].values
df["unique_number_of_SH_involved_per_year"] = df_inv_unq.iloc[:,0].values
df["total_transaction_number_per_year"] = df_inv.iloc[:,0].values
df

,entid,Vol/transaction_per_year,net_change_per_year,unique_number_of_SH_involved_per_year,total_transaction_number_per_year
0,243165,243165,0.621989,2.0,2.0
1,683429,683429,-103.595820,2.0,2.0
2,703531,703531,1.930562,1.0,1.5
3,2126661,2126661,0.262459,2.0,2.0
4,3926205,3926205,-4.049364,2.0,2.0
...,...,...,...,...,...
1175,1223416010,1223416010,-4.278679,5.0,5.0
1176,1225572865,1225572865,5.526924,3.0,3.0
1177,1227102837,1227102837,-49.806573,1.0,1.0
1178,1228040547,1228040547,0.191677,3.0,3.0


In [33]:
df.to_csv(outpath+"company_ar_alterstockinfo_processed.csv",index=False)